# A (rapid) Introduction to Python & SunPy
## Stuart Mumford
### Aperio Software Ltd.

This session will give you a quick introduction to doing solar and heliospheric physics with Python, and a quick demo.

In the last two hour slot of your week, I am not going to be able to teach you Python, but hopefully I can show you that there are lots of tools to help you and give you enough information that you can find help once you leave here!

## Why Python?

# Community

# Employability

Sure there are loads of other reasons, but Python has loads of scientific and Solar / Helio specific functionality, and on top of that it has a welcoming and helpful community and because it's a programming language used all over the world for a load of different things, it's a great skill to have.

# The SunPy Community

Like many other scientific Python projects, SunPy's packages are **"Open Development"** this means anyone is encouraged to contribute in any way that they can, this might include:


* Telling the developers when something doesn't work, a feature is missing or hard to understand.
* Telling the developers what worked well for you, what you found easy to understand.
* Helping other people in the community by answering questions.
* Improving the documentation, adding examples.
* Fixing bugs and adding new features.

We *want you* to help us make scientific Python better!

# Modules, modules everywhere...

Python is a modular language, different people and groups develop _"packages"_ which are distrubuted and installable.

You pick the set of packages which help you solve your problems, and combine them together.

`sunpy`, `astropy`, `numpy` and `heliopy` are all examples of packages which you can install and then "import".

# An Exercise

Let's spend most of the rest of this session actually writing some code.

If you haven't used Python at all before some of this might be a little complex, but I will try and explain as we go along, we are going to play with a toy example of tracing a feature in the solar wind back to the Sun.
This is going to be *super* simplified, but the idea is to show off a few Python things in a couple of hours, not actually draw any scientific conclusions!

Let's start by downloading some in-situ data from [OMNIWeb](https://omniweb.gsfc.nasa.gov/html/ow_data.html) which is an aggregated data set of near-Earth observations.

For this we will use the downloader in [`heliopy`](https://docs.heliopy.org/en/stable/) which returns a [sunpy `Timeseries` object](https://docs.sunpy.org/en/stable/guide/data_types/timeseries.html).

In [ ]:
from heliopy.data import omni

In [ ]:
from sunpy.time import parse_time

In [ ]:
omni_data = omni.hro2_1min(starttime=parse_time("2018-10-01").datetime,
                           endtime=parse_time("2018-11-01").datetime)

In [ ]:
omni_data

In [ ]:
omni_data.meta

In [ ]:
for col in omni_data.columns:
    print(col)

In [ ]:
omni_data.units

Plotting in-situ data
---
Matplotlib can be used to plot the downloaded data. In this example we plot the solar wind speed and the z component of the magnetic field, to see different polarity solar wind streams.

In [ ]:
%matplotlib widget

In [ ]:
import matplotlib.pyplot as plt

import astropy.units as u
from astropy.visualization import quantity_support
quantity_support()

In [ ]:
fig, axs = plt.subplots(nrows=3, figsize=(9, 6), sharex=True)

ax = axs[0]
ax.plot(omni_data.index, omni_data.quantity("flow_speed"), label="$v_{sw}$")

ax = axs[1]
ax.plot(omni_data.index, omni_data.quantity("BZ_GSE"), label="$B_z$")

ax = axs[2]
ax.plot(omni_data.index, omni_data.quantity("proton_density"), label="$n_p$")

## Finding Maxima (the easy way)

To give us a single time to track back the flow to the Sun, let's find the point in this months worth of data which had the highest flow speed.

If you were doing this properly then you would want to smooth the data, which you could do in many ways such as the [Savitzky-Golay filter](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.savgol_filter.html) in `scipy`. For expedience we are just going to find the maxima.

In [ ]:
import numpy as np

In [ ]:
# Extract a unit-aware array
flow_speed = omni_data.quantity('flow_speed')

In [ ]:
flow_speed

In [ ]:
np.max(flow_speed)

In [ ]:
flow_speed[np.isnan(flow_speed)] = 0

In [ ]:
vel_max = np.max(flow_speed)
vel_max

In [ ]:
vel_max_index = np.argmax(flow_speed)
vel_max_index

In [ ]:
vel_max_time = omni_data.index[vel_max_index]
vel_max_time

Improving figure formatting
---

We can improve this plot a little with some better formatting, and add a vertical line for our maximum flow speed.

In [ ]:
fig.autofmt_xdate()
fig.subplots_adjust(hspace=0)

for ax in axs:
    ax.legend()
    ax.axvline(vel_max_time, color='black', linewidth=1, linestyle='--')

# Add a zero line to the Bz plot
axs[1].axhline(0, color='black', linewidth=1)
    
fig

# Tracing the flow back to the Sun

Let's figure out, in a very inaccurate manner, rougly when this solar wind stream would have left the Sun.

To calculate this propagation delay we can use the ``astropy.units`` module. This provides an extension of normal numbers and arrays, and allows units to be attached. All the unit mathematics is calculated automatically, avoiding the need to keep track of specific units.

In [ ]:
import astropy.constants as const
from sunpy.coordinates.ephemeris import get_earth

Let's assume the solar wind is relesed from the surface of the Sun, so the propagation distance is $D_{sun} - R_{sun}$

In [ ]:
d_sun = get_earth(vel_max_time)
d_sun

In [ ]:
d = (d_sun.radius - const.R_sun)
d

Let's assume that this stream would have been travelling at the speed it arrived at Earth the whole time:

In [ ]:
vel_max

In [ ]:
vel_max.to(u.imperial.mile/u.hour)

Calculate tne propagation time, and convert it to units of days

In [ ]:
t = (d / vel_max).to(u.day)
t

In [ ]:
estimated_departure = parse_time(vel_max_time) - t
estimated_departure

# Downloading Data with sunpy's Fido

Now we have a rough idea of the time that the wind stream would have left the Sun, let's see what was happening in EUV around that time.

In [ ]:
start_time = estimated_departure - 1 * u.day
end_time = estimated_departure + 1 * u.day

In [ ]:
from sunpy.net import Fido, attrs as a

In [ ]:
result = Fido.search(a.Time(start_time, end_time),
                     a.Sample(3 * u.hour),
                     a.Instrument.aia,
                     a.Wavelength(19.3 * u.nm))
result

In [ ]:
files = Fido.fetch(result)

In [ ]:
files

## sunpy Map

Having downloaded some AIA data, let us load it with `sunpy`.

`sunpy` provides a [`Map` datatype](https://docs.sunpy.org/en/stable/guide/data_types/maps.html) which provides a coordinate aware wrapper to 2D imaging data, and sequences of such images.

To start with we will load all the images we just downloaded into a single [`MapSequence`](https://docs.sunpy.org/en/stable/api/sunpy.map.MapSequence.html) object.

In [ ]:
import sunpy.map

In [ ]:
map_seq = sunpy.map.Map(files, sequence=True)

In [ ]:
map_seq.peek()

# Picking out a single image

To show off `Map` a little more, let us pick out the map which is closest in time to our estimated time. To do this we subtract the map's time from the estimated time and find the minima of the absolute difference.

In [ ]:
map_time_shift = [estimated_departure - m.date for m in map_seq]

In [ ]:
map_time_shift

In [ ]:
nearest_map_index = np.argmin(np.abs(map_time_shift))
nearest_map_index

In [ ]:
nearest_map = map_seq[nearest_map_index]
nearest_map

In [ ]:
from astropy.coordinates import SkyCoord

In [ ]:
plt.figure()
ax = plt.subplot(projection=nearest_map)
nearest_map.plot()
nearest_map.draw_grid()

ax.plot_coord(SkyCoord(-60*u.deg, -5*u.deg, frame="heliographic_stonyhurst"), marker='o')